来源：http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html

## 简介

神经网络中一些需要确定的超参数：
> 1. Number of layers  
> 2. Different parameters for each layer (number of hidden units, filter size for convolutional layer and so on)  
> 3. Type of activation functions  
> 4. Parameter initialization method  
> 5. Learning rate  
> 6. Loss function  

一些超参数选择方法： 
> 1. Grid Search  
> 2. Random Search  
> 3. Hand-tuning 
> 4. Gaussian Process with Expected Improvement  
> 5. Tree-structured Parzen Estimators (TPE)  

其中4，5属于Bayesian optimization。

## Grid Search

为每个要确定的参数定义一组值，依次尝试所有的组合。   
5个参数，每个参数10个可能值，一共$10^5$种组合。  
耗时会很久。

## Random Search

随机地挑选一些超参数。  
但随机挑选的结果将是点的分布有疏有密。  
可以利用low-discrepancy sequence缓解这个问题，比如Halton sequence。  
但是维度高时，这种方法并不理想。

![p1](./100-uniform-data-points.png)

![p2](./quasi-random-points.PNG)

## Hand-tuning

人在调参的过程中可以记住并分析之前的结果，在之后的超参数设置时可以利用这些经验。  
在这个角度，hand-tuning比grid search和random search高效，因为后面两种方法没有记忆。

如何存储记忆？有深度的方法和非深度的方法。深度方法比如使用RL，非深度的方法比如下面提到的Bayesian optimization。

## Bayesian Optimization

将上述过程自动化，可以得到Bayesian optimization。  
在Bayesian optimization中可以利用Gaussian Process与Acquisition Function。  
Gaussian Process可以利用已有的调参结果建立关于超参数的假设，Acquisition Function可以利用这些知识进行下一轮的选择。

### Gaussian Process

Gaussian Process：对每个x有 y=f(x)  
f是一个随机函数，它从一个与x有关的高斯分布采样出输出y。

![p3](./guassian-process-1.PNG)

随着采样的不断进行，这个函数会越来越清晰。（下图蓝色区域为95%置信区间）

![p4](./guassian-process-2.PNG)

#### Acquisition Function

Acquisition function可以获得当前对超参数的分布的估计下的最优解。

Expected Improvement（期望提升）的定义：$g_{max}(x)=max(0,y_{highest}-y_{max})$  
$y_{highest}$为当前分布的最高点，  
$y_{max}$为观测值最高点。

![p5](./expected-improvement-example.PNG)

下面的例子中使用基于Gaussian process和expected improvement的Bayesian optimization来寻找最佳的隐层单元数量。

In [ ]:
# 下面的函数可以设置隐层单元数量，然后训练网络，返回错误率

from neupy import algorithms, layers

def train_network(n_hidden, x_train, x_test, y_train, y_test):
    network = algorithms.Momentum(
        [
            layers.Input(64),
            layers.Relu(n_hidden),
            layers.Softmax(10),
        ],

        # Randomly shuffle dataset before each
        # training epoch.
        shuffle_data=True,

        # Do not show training progress in output
        verbose=False,

        step=0.001,
        batch_size=128,
        error='categorical_crossentropy',
    )
    network.train(x_train, y_train, epochs=100)

    # Calculates categorical cross-entropy error between
    # predicted value for x_test and y_test value
    return network.prediction_error(x_test, y_test)

In [ ]:
# 准备数据集

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from neupy import environment

environment.reproducible()

dataset = datasets.load_digits()
n_samples = dataset.target.size
n_classes = 10

# One-hot encoder
target = np.zeros((n_samples, n_classes))
target[np.arange(n_samples), dataset.target] = 1

x_train, x_test, y_train, y_test = train_test_split(
    dataset.data, target, train_size=0.7
)

In [ ]:
# 下面的函数利用观测到的数据（x_train，y_train）来拟合一个gaussian process
# 然后为每个可能的隐层个数生成对应的均值和标准差，然后返回

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor

def vector_2d(array):
    return np.array(array).reshape((-1, 1))

def gaussian_process(x_train, y_train, x_test):
    x_train = vector_2d(x_train)
    y_train = vector_2d(y_train)
    x_test = vector_2d(x_test)

    # Train gaussian process
    gp = GaussianProcessRegressor(corr='squared_exponential',
                         theta0=1e-1, thetaL=1e-3, thetaU=1)
    gp.fit(x_train, y_train)

    # Get mean and standard deviation for each possible
    # number of hidden units
    y_mean, y_var = gp.predict(x_test, eval_MSE=True)
    y_std = np.sqrt(vector_2d(y_var))

    return y_mean, y_std

In [ ]:
# 这个函数先计算每个可能隐层个数的EI，然后返回使得EI最大的隐层数量

def next_parameter_by_ei(y_min, y_mean, y_std, x_choices):
    # Calculate expecte improvement from 95% confidence interval
    expected_improvement = y_min - (y_mean - 1.96 * y_std)
    expected_improvement[expected_improvement < 0] = 0

    max_index = expected_improvement.argmax()
    # Select next choice
    next_parameter = x_choices[max_index]

    return next_parameter

In [ ]:
# 通过若干轮 gaussian process估计--EI选择 来找最佳参数

import random

def hyperparam_selection(func, n_hidden_range, func_args=None, n_iter=20):
    if func_args is None:
        func_args = []

    scores = []
    parameters = []

    min_n_hidden, max_n_hidden = n_hidden_range
    n_hidden_choices = np.arange(min_n_hidden, max_n_hidden + 1)

    # To be able to perform gaussian process we need to
    # have at least 2 samples.
    n_hidden = random.randint(min_n_hidden, max_n_hidden)
    score = func(n_hidden, *func_args)

    parameters.append(n_hidden)
    scores.append(score)

    n_hidden = random.randint(min_n_hidden, max_n_hidden)

    for iteration in range(2, n_iter + 1):
        
        print("n_hidden:", n_hidden)
        
        score = func(n_hidden, *func_args)

        parameters.append(n_hidden)
        scores.append(score)

        y_min = min(scores)
        y_mean, y_std = gaussian_process(parameters, scores,
                                         n_hidden_choices)

        n_hidden = next_parameter_by_ei(y_min, y_mean, y_std,
                                        n_hidden_choices)

        if y_min == 0 or n_hidden in parameters:
            # Lowest expected improvement value have been achieved
            break

    min_score_index = np.argmin(scores)
    return parameters[min_score_index]

In [ ]:
# 运行

best_n_hidden = hyperparam_selection(
    train_network,
    n_hidden_range=[50, 1000],
    func_args=[x_train, x_test, y_train, y_test],
    n_iter=6,
)

#### GP with EI 的一些缺点：

> 1. 不能很好地处理一些类别数据，比如激活函数的类型  
> 2. GP本身需要设置一些超参数  
> 3. 当超参数数量多时会很慢  
> 4. NN运行中会有一些随机化操作（如dropout），估计的最优未必最优

### Tree-structured Parzen Estimators (TPE)

注意！GP法是直接对p(y|x)进行建模，即给定x，建模y服从的分布；而TPE法则是对p(x|y)进行建模，即给定y的条件下，建模x服从的分布。

![p](./TPE-model.PNG)

> 1. 开始的几次迭代，需要若干次random search来warm up TPE   
> 2. 在收集一些数据后，就可以运用TPE。要将所有observations分成两部分，第一部分具有好效果，第二部分是其他observations  

![p6](./tpe-observation-groups.PNG)

之后便不再需要具体的best observations，只要其分布。

随后TPE将为两组建立各自的似然概率（GP构建的是各值的后验概率）

于是Expected Inprovement的定义：$EI(x)=\frac{l(x)}{g(x)}$  
$l(x)$是x属于第一组的概率  
$g(x)$是x属于第二组的概率

![p7](./tpe-sampled-candidates.PNG)

l(x)与g(x)的估计方法：parzen-window density estimators  
每个样本点定义了一个具有特定均值和标准差的高斯分布，将某组的所有点的分布堆叠起来，再归一化，便可以得到该类的pdf。

![p8](./parzen-estimators.PNG)

tree-structured是指超参数空间以树的形式组织。

GP法不能支持类别数据，因为GP法本质上要拟合出gaussian process的函数，这要求定义域连续；而TPE用密度估计的方法，就不需要连续，此时EI所选的点不会落在两个类别之间。

下面使用MNIST数据集来举个TPE算法的例子：

使用hyperopt库进行超参数的选择，它实现了TPE算法

In [ ]:

from pprint import pprint
from functools import partial

import theano
import numpy as np
from sklearn import model_selection, datasets, preprocessing, metrics
import hyperopt
from hyperopt import hp
from neupy import algorithms, layers, environment
from neupy.exceptions import StopTraining

theano.config.floatX = 'float32'

def on_epoch_end(network):
    if network.errors.last() > 10:
        raise StopTraining("Training was interrupted. Error is to high.")

def train_network(parameters):
    print("Parameters:")
    pprint(parameters)
    print()

    step = parameters['step']
    batch_size = int(parameters['batch_size'])
    proba = parameters['dropout']
    activation_layer = parameters['act_func_type']
    layer_sizes = [int(n) for n in parameters['layers']['n_units_layer']]
    
    network = layers.Input(784)

    for layer_size in layer_sizes:
        network = network > activation_layer(layer_size)

    network = network > layers.Dropout(proba) > layers.Softmax(10)
    
    mnet = algorithms.RMSProp(
        network,

        batch_size=batch_size,
        step=step,
        
        error='categorical_crossentropy',
        shuffle_data=True,
        
        epoch_end_signal=on_epoch_end,
    )
    mnet.train(x_train, y_train, epochs=50)
    
    score = mnet.prediction_error(x_test, y_test)
    
    y_predicted = mnet.predict(x_test).argmax(axis=1)
    accuracy = metrics.accuracy_score(y_test.argmax(axis=1), y_predicted)
    
    print("Final score: {}".format(score))
    print("Accuracy: {:.2%}".format(accuracy))

    return score

In [ ]:
# 构造超参数的先验分布

def uniform_int(name, lower, upper):
    # `quniform` returns:
    # round(uniform(low, high) / q) * q
    return hp.quniform(name, lower, upper, q=1)

def loguniform_int(name, lower, upper):
    # Do not forget to make a logarithm for the
    # lower and upper bounds.
    return hp.qloguniform(name, np.log(lower), np.log(upper), q=1)

In [ ]:
def load_mnist_dataset():
    mnist = datasets.fetch_mldata('MNIST original')

    target_scaler = preprocessing.OneHotEncoder()
    target = mnist.target.reshape((-1, 1))
    target = target_scaler.fit_transform(target).todense()

    data = mnist.data / 255.
    data = data - data.mean(axis=0)

    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        data.astype(np.float32),
        target.astype(np.float32),
        train_size=(6 / 7.)
    )
    
    return x_train, x_test, y_train, y_test

environment.reproducible()
x_train, x_test, y_train, y_test = load_mnist_dataset()

In [ ]:
# 构造超参数的先验分布

# Object stores all information about each trial.
# Also, it stores information about the best trial.
trials = hyperopt.Trials()

parameter_space = {
    'step': hp.uniform('step', 0.01, 0.5),
    'layers': hp.choice('layers', [{
        'n_layers': 1,
        'n_units_layer': [
            uniform_int('n_units_layer_11', 50, 500),
        ],
    }, {
        'n_layers': 2,
        'n_units_layer': [
            uniform_int('n_units_layer_21', 50, 500),
            uniform_int('n_units_layer_22', 50, 500),
        ],
    }]),
    'act_func_type': hp.choice('act_func_type', [
        layers.Relu,
        layers.PRelu,
        layers.Elu,
        layers.Tanh,
        layers.Sigmoid
    ]),
    
    'dropout': hp.uniform('dropout', 0, 0.5),
    'batch_size': loguniform_int('batch_size', 16, 512),
}

In [ ]:
tpe = partial(
    hyperopt.tpe.suggest,

    # Sample 1000 candidate and select candidate that
    # has highest Expected Improvement (EI)
    n_EI_candidates=1000,
    
    # Use 20% of best observations to estimate next
    # set of parameters
    gamma=0.2,
    
    # First 20 trials are going to be random
    n_startup_jobs=20,
)

hyperopt.fmin(
    train_network,
    trials=trials,
    space=parameter_space,

    # Set up TPE for hyperparameter optimization
    algo=tpe,

    # Maximum number of iterations. Basically it trains at
    # most 200 networks before choose the best one.
    max_evals=200,
)